In [5]:
from pyod.models.iforest import IForest
from sklearn.utils import shuffle
from pysad.evaluation import AUROCMetric
from pysad.models.integrations import ReferenceWindowModel
from pysad.utils import ArrayStreamer
from pysad.utils import Data
from tqdm import tqdm
import numpy as np
import pandas as pd

In [13]:
data = pd.read_csv("Data/Edu_TS_2_Traffic.csv", sep=";", parse_dates=True)
data['Date'] = pd.to_datetime(data['Date'], format='%d.%m.%Y')
data.sort_values(by='Date', inplace=True)
iterator = ArrayStreamer(shuffle=False)

In [14]:
data_X = data['Unique Visitors'].values.reshape(-1, 1)  # Reshape to ensure 2D array
data_Y = None

In [15]:
model = ReferenceWindowModel(model_cls=IForest, window_size=24, sliding_size=3, initial_window_X=data_X[:10])
auroc = AUROCMetric()

In [16]:
for X, y in tqdm(iterator.iter(data_X[10:], data_Y[10:])):

    model.fit_partial(X)  # Fit to the instance.
    score = model.score_partial(X)  # Score the instance.

    auroc.update(y, score)  # Update the metric.

# Output AUROC metric.
print("AUROC: ", auroc.get())

TypeError: 'NoneType' object is not subscriptable